# Apply to financial applications

The present notebook will use the the library for a finance application. Specifically we want to solve a pricing problem. The reference for this notebook will be:

* NEASQC deliverable: *D5.1: Review of state-of-the-art for Pricing and Computation of VaR https://www.neasqc.eu/wp-content/uploads/2021/06/NEASQC_D5.1_Review-of-state-of-the-art-for-Pricing-and-Computation-of-VaR_R2.0_Final.pdf*

Given an input function $f(x)$ and a proability density $p(x)$ we want to compute the expected value of  $f(x)$ when x follows $p(x)$:

$$\mathbb{E}[f]=\int_a^bp(x)f(x)dx$$

This integral can be approximated by the Riemann sum:

$$\mathbb{E}[f] = \sum_{i=0}^{2^n-1} p(x_i)f(x_i)dx$$

The idea is to use **Amplitude Estimation** procedure in order to compute this value.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm
from copy import deepcopy

In [ ]:
import sys
sys.path.append("../")

In [ ]:
%matplotlib inline

In [ ]:
#This cell loads the QLM solver.
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg (for using QPU as CESGA QLM one)
from libraries.utils.qlm_solver import get_qpu
QLMaaS = False
linalg_qpu = get_qpu(QLMaaS)

In [ ]:
#See 01_DataLoading_Module_Use for the use of this function
from libraries.utils.data_extracting import get_results

## 1. Discretizing data

Below cell defines the functions for $p(x)$ and $f(x)$

In [ ]:
def p(x):
    return x*x
def f(x):
    return np.sin(x)

First thing to do is define a discrete version of these functions. We can use the *get_histogram* function from *utils/utils* module

In [ ]:
from libraries.utils.utils import get_histogram

In [ ]:
#number of Qbits for the circuit
n_qbits = 4

In [ ]:
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

It is mandatory that $p(x)$ and $f(x)$ were properly normalized. Following conditions should be fullfilled:

* For $p(x)$ is mandatory that: $\sum_{i=0}^{2^{n}} p(x_i) = 1$
* For $f(x)$ is mandatory that: $f(x_i) \leq 1 \forall i$

In [ ]:
%matplotlib inline
plt.plot(X, p_X, 'o')
plt.plot(X, f_X, 'o')
plt.legend(['p(x)', 'f(x)'])

In [ ]:
#Test condition in p(x)
print('p(x) condition: {}'.format(np.sum(p_X) == 1))
print('f(x) condition: {}'.format(np.max(f_X) <= 1))

## 2. Loading data

Second part is loading the data ($p(x)$ and $f(x)$) into the quantum state. For this we need to create following 2 loading operators:

1. $\mathcal{P}$. for loading $p(x)$
2. $\mathcal{F}$. for loading $f(x)$

So the main idea is begin with an initial $n+1$ qbits state:

$$|0\rangle \otimes|0\rangle_{n}$$

An apply the following loading operation:

$$|\Psi\rangle=\mathcal{F}\left(I\otimes\mathcal{P}\right)|0\rangle\otimes|0\rangle_{n}$$

The quantum state resulting from the loading proccess: $|\Psi\rangle$ can be expressed as a linear combination of two orthogonal states $|\Psi_{1}\rangle$ y $|\Psi_{0}\rangle$:

$$|\Psi\rangle=\sqrt{a}|\Psi_{0}\rangle+\sqrt{1-a}|\Psi_{1}\rangle$$

where:
$$
\begin{array}{l}
&\sqrt{a}|\Psi_{0}\rangle = |0\rangle \otimes\sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)f(x_i)}|i\rangle_{n},\\\\
&\sqrt{1-a}|\Psi_{1}\rangle = |1\rangle\otimes\sum_{i=0}^{2^{n}-1}\sqrt{p(x_i)(1-f(x_i))}|i\rangle_{n}.
\end{array}
$$
The probability of measuring $|0\rangle$ is:

$$ \left|\sqrt{a}\right|^2 = \sum_{i=0}^{2^{n}-1}\left|\sqrt{p(x_i)f(x_i)}\right|^2,$$

so estimating $\sqrt{a}$ gives as an approximation of the value of the integral.

For doing this operations *load_probability* and *load_array* functions from **DL/data_loading** module will be used (see notebook **01_DataLoading_Module_Use.ipynb**)

In [ ]:
from libraries.DL.data_loading import load_probability, load_array

In [ ]:
p_gate = load_probability(p_X)
f_gate = load_array(np.sqrt(f_X))

In [ ]:
%qatdisplay p_gate --depth 0
%qatdisplay f_gate --depth 0

Now we compound the two operators $\mathcal{P}$ aqnd $\mathcal{F}$ for creating the necesary loading circuit

In [ ]:
routine = qlm.QRoutine()
register = routine.new_wires(f_gate.arity)
routine.apply(p_gate, register[:p_gate.arity])
routine.apply(f_gate, register)

In [ ]:
%qatdisplay routine --depth 0

We can test if the loading was properly implemented. For doing this we are going to use **get_results** from *data_extracting* module

In [ ]:
results,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)
quantum_probabilities = results["Probability"].values

In [ ]:
np.isclose(quantum_probabilities[0:2**p_gate.arity], p_X*f_X).all()

We are going to measure the complete state (**phi_state**) and the last qbit state (**InitialState**)

In [ ]:
phi_state,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu)#, qubits=[f_gate.arity-1])
InitialState,_,_,_ = get_results(routine, linalg_qpu=linalg_qpu, qubits=[f_gate.arity-1])

In [ ]:
np.isclose(InitialState['Probability'].iloc[0], sum(p_X*f_X))

## 3. Maximum Likelihood Amplitude Estimation

Quantum circuit implementation of $\mathcal{QFT}$ are very complex and very long and deep so its use in actual quantum computers is noisy and not very useful.

The **Maximum Likelihood Amplitude Estimation** algorithm (**MLAE**) allows to take advantage of **amplitude amplification** without the use of the  $\mathcal{QFT}$.

For doing this  we are going to use the  **maximum_likelihood_ae** module 

In [ ]:
from  libraries.AE.maximum_likelihood_ae  import MLAE

The notebook **04_MaximumLikelihood_Class** explains how this class works. In our case we need to provide:

1. Oracle
2. Target: the binnary representation of the marked state. 
3. index: the registers where the Target is.

In our case is the same that we use for generating Grover-like operator

In [ ]:
oracle = deepcopy(routine)
target = [0]
index = [routine.arity-1]

In [ ]:
%qatdisplay oracle

Additionally a schedule where the list of applications of Grover-like operator should be provided

In [ ]:
m_k = list(range(5))
n_k = [100]*len(m_k)
schedule = [m_k,n_k]

In [ ]:
mlae = MLAE(
    oracle,target = target,
    index = index,
    schedule = schedule)

In [ ]:
mlae.schedule

In [ ]:
angle = mlae.optimize()

In [ ]:
#ploting the Cost function
angles = np.linspace(0+mlae.delta,np.pi/2-mlae.delta,100)
cost = np.zeros(len(angles))
for i in range(len(angles)):
    cost[i] = mlae.cost_function(angles[i])

plt.plot(angles, cost)
plt.axvline(np.arcsin(sum(p_X*f_X)**0.5), color = 'r',label = r"$\theta^*$")

#plt.axvline(theta,color = 'r')
plt.grid()
plt.xlabel(r"$\theta$")
plt.ylabel(r"$C(\theta)$")

plt.legend()

In this case the aproximation of the integral $\mathbb{E}[f]\approx \sum_{i=0}^{2^{n}-1}\left|\sqrt{p(x_i)f(x_i)}\right|^2 $ aproximated as:

$$\sin^2(\theta),$$ 

where $\theta$ is the angle calculated with the **MLAE** algortihm. See notebook **03_MaximumLikelihood_Class** for more information. 

In [ ]:
print("Quantum result: ",np.sin(angle)[0]**2)
print('Integral Result: ', sum(p_X*f_X))
print('Difference: ', abs(np.sin(angle)[0]**2-sum(p_X*f_X)))

## 4. Kitaev's Iterative Quantum Amplitude Estimation

Another alternative to estimate the amplitude is using  *Iterative Quantum Phase Estimation* to do *Quantum Amplitude Estimation*.
For using **IterativeQuantumPE** we need to provide:

* initial_state : in this case will be the oracle
* unitary_operator:  in this case will be the Grover-like operator build using *oracle* 
* cbits_number: number of classical bits used for phase estimation

For creating the grover operator we are going to use the *grover* function from *amplitude_amplification* module and the same *target* and *index*  then in the *maximum likelihood* class will be used:

In [ ]:
from libraries.AA.amplitude_amplification import grover

In [ ]:
target = [0]
index = [oracle.arity-1]
q_gate = grover(oracle, target, index)

In [ ]:
%qatdisplay q_gate --depth 1

In [ ]:
#Load Class
from libraries.PE.iterative_quantum_pe import IterativeQuantumPE

In [ ]:
n_cbits = 7
#We create a python dictionary for configuration of class
iqpe_dict = {
    'initial_state': oracle,
    'unitary_operator': q_gate,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,
    'shots': 100
    #'easy': True,
    #'easy': False    
}
IQPE = IterativeQuantumPE(**iqpe_dict)

In [ ]:
IQPE.iqpe()

In [ ]:
IQPE.final_results

In this case the aproximation of the integral $\mathbb{E}[f]\approx \sum_{i=0}^{2^{n}-1}\left|\sqrt{p(x_i)f(x_i)}\right|^2 $ aproximated as:

$$\cos^2(\theta),$$ 

where $\theta$ is the angle calculated with the **IQPE** algortihm. See notebook **04_Iterative_QPE_Class** for more information. 

In [ ]:
pdf = IQPE.sumary
pdf['P']=np.cos(pdf['theta_90'])**2

In [ ]:
pdf

In [ ]:
print("Classical result: ", np.sum(p_X*f_X))
print("Quantum result: ",pdf.iloc[0]['P'])